### P.s. siate sempre gentili quando fate richieste alle macchine, quando conquisteranno il mondo, magari si ricorderanno di voi e vi risparmieranno 😂

In [10]:
!git clone https://github.com/Frenz86/tutotrialgpt4-o.git

Cloning into 'tutotrialgpt4-o'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 31 (delta 10), reused 24 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 7.29 MiB | 16.66 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [11]:
cd tutotrialgpt4-o

/content/tutotrialgpt4-o/tutotrialgpt4-o


In [12]:
!pip install -r requirements.txt -q

### API CHATGPT-4o Part1

In [13]:
!pip install openai -q

In [ ]:
# not in colab
# import os
# api_key = os.environ.get('OPENAI_API_KEY')

In [14]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [21]:
from openai import OpenAI

#MODEL="gpt-4o"
MODEL="gpt-4o-mini"

#client = OpenAI(api_key="sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
client = OpenAI()

### 1 - Basic Chat

In [20]:
richiesta = input('inserisci richiesta ')
completion = client.chat.completions.create(
                                            model=MODEL,
                                            messages=[
                                                {"role": "system", "content": "Sei un assistente per lo studio. Cortesemente aiutami con i compiti"},
                                                {"role": "user", "content": richiesta}
                                            ]
                                            )

print("Assistante: " + completion.choices[0].message.content)

inserisci richiesta che ore sono?
Assistante: Non posso fornirti l'ora attuale, ma puoi facilmente controllarla sul tuo dispositivo. Ti serve aiuto con qualcos'altro?


### 2 - Image Processing: Base64


In [22]:
import base64

IMAGE_PATH = "triangle.png"

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

base64_image = encode_image(IMAGE_PATH)

response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": "Sei un assistente per lo studio. Cortesemente aiutami con i compiti"},
                {"role": "user", "content": [
                    {"type": "text", "text": "Quanto vale l'area del triangolo, svolgi i passaggi e dammi il risultato"},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}
                    }
                ]}
            ],
            temperature=0.0,
        )

print(response.choices[0].message.content)

Per calcolare l'area del triangolo, possiamo utilizzare la formula:

\[
\text{Area} = \frac{1}{2} \times \text{base} \times \text{altezza}
\]

Nel triangolo che hai fornito, la base è 9 e l'altezza è 6. Sostituiamo i valori nella formula:

\[
\text{Area} = \frac{1}{2} \times 9 \times 6
\]

Calcoliamo:

\[
\text{Area} = \frac{1}{2} \times 54 = 27
\]

Quindi, l'area del triangolo è **27** unità quadrate.


### 3 - Image Processing: URL


In [ ]:
url = "https://upload.wikimedia.org/wikipedia/commons/e/e2/The_Algebra_of_Mohammed_Ben_Musa_-_page_82b.png"

response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": "Sei un assistente per lo studio. Cortesemente aiutami con i compiti"},
                    {"role": "user", "content": [
                        {"type": "text", "text": "Quanto vale l'area del triangolo, svolgi i passaggi e dammi il risultato"},
                        {"type": "image_url", "image_url": {"url":f"{url}"}
                        }
                    ]}
                ],
                temperature=0.0,
                )

print(response.choices[0].message.content)

Per calcolare l'area del triangolo, possiamo utilizzare la formula dell'area di un triangolo generico:

\[ A = \frac{1}{2} \times base \times altezza \]

Tuttavia, in questo caso, non conosciamo direttamente l'altezza del triangolo. Possiamo calcolarla utilizzando il teorema di Pitagora. 

Il triangolo è diviso in due triangoli rettangoli dalla sua altezza. La base del triangolo è 9, quindi ogni metà della base è 4.5. I lati dei due triangoli rettangoli sono 6 e 5.

Per il triangolo con ipotenusa 6 e base 4.5, possiamo calcolare l'altezza \(h\) usando il teorema di Pitagora:

\[ 6^2 = 4.5^2 + h^2 \]

\[ 36 = 20.25 + h^2 \]

\[ h^2 = 36 - 20.25 \]

\[ h^2 = 15.75 \]

\[ h = \sqrt{15.75} \]

\[ h \approx 3.97 \]

Ora possiamo calcolare l'area del triangolo:

\[ A = \frac{1}{2} \times base \times altezza \]

\[ A = \frac{1}{2} \times 9 \times 3.97 \]

\[ A \approx \frac{1}{2} \times 9 \times 3.97 \]

\[ A \approx 17.865 \]

Quindi, l'area del triangolo è approssimativamente 17.865 unità q

### 4 - Summarization: Video Summary
Lo script permette di elaborare un video estraendo frame a intervalli regolari e l'audio associato al video

In [ ]:
import cv2
from moviepy.editor import VideoFileClip
import base64
import os

VIDEO_PATH = "keynote_recap.mp4"

def process_video(video_path, seconds_per_frame=2):
    base64Frames = []
    base_video_path, _ = os.path.splitext(video_path)

    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)
    frames_to_skip = int(fps * seconds_per_frame)
    curr_frame = 0

    while curr_frame < total_frames - 1:
        video.set(cv2.CAP_PROP_POS_FRAMES, curr_frame)
        success, frame = video.read()
        if not success:
            break
        _, buffer = cv2.imencode(".jpg", frame)
        base64Frames.append(base64.b64encode(buffer).decode("utf-8"))
        curr_frame += frames_to_skip
    video.release()

    audio_path = f"{base_video_path}.mp3"
    clip = VideoFileClip(video_path)
    clip.audio.write_audiofile(audio_path, bitrate="32k")
    clip.audio.close()
    clip.close()

    print(f"Extracted {len(base64Frames)} frames")
    print(f"Extracted audio to {audio_path}")
    return base64Frames, audio_path

def submit_to_gpt(base64Frames, model):
    max_frames_per_request = 10  # Adjust based on the token limit
    for i in range(0, len(base64Frames), max_frames_per_request):
        batch_frames = base64Frames[i:i + max_frames_per_request]
        messages = [
                    {"role": "system", "content": "Stai generando un riepilogo del video, si prega di fornire un riepilogo del video in formato testuale Markdown"},
                    {"role": "user", "content": [
                        "Questi sono i frame del video",
                        *map(lambda x: {"type": "image_url", "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}}, batch_frames)
                    ]}
                    ]

        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        temperature=0,
                        )

        print(response.choices[0].message.content)

base64Frames, audio_path = process_video(VIDEO_PATH, seconds_per_frame=1)
submit_to_gpt(base64Frames, MODEL)

MoviePy - Writing audio in keynote_recap.mp3


MoviePy - Done.
Extracted 218 frames
Extracted audio to keynote_recap.mp3
Ecco un riepilogo del video basato sui frame forniti:

---

## Riepilogo del Video

### OpenAI DevDay

Il video inizia con una serie di schermate che mostrano il titolo "OpenAI DevDay". Successivamente, appare la scritta "Keynote Recap", indicando che il video fornirà un riassunto della presentazione principale dell'evento.

### Scene dell'Evento

1. **Ingresso dell'Evento**: Viene mostrato l'ingresso dell'evento con un'insegna che riporta "OpenAI DevDay".
2. **Logo di OpenAI**: Un'insegna con il logo di OpenAI è visibile, suggerendo che l'evento è ufficialmente organizzato da OpenAI.
3. **Interno dell'Evento**: Scene all'interno della sede mostrano partecipanti che si muovono e interagiscono, creando un'atmosfera vivace e dinamica.
4. **Sala Conferenze**: Una sala conferenze con sedie disposte e uno schermo grande sullo sfondo, pronta per ospitare le presentazioni e le discussioni.

---

Questo riepilogo fornisc

### 5 - Summarization: Audio Summary


In [ ]:
audio_path = "keynote_recap.mp3"

transcription = client.audio.transcriptions.create(
                    model="whisper-1",
                    file=open(audio_path, "rb"),
                    )

transcription_text = transcription.text
with open("transcription.txt", "w") as file:
    file.write(transcription_text)

response = client.chat.completions.create(
                model=MODEL,
                messages=[
                {"role": "system", "content":"""Stai generando un riepilogo della trascrizione.
                                                Crea il riepilogo della trascrizione fornita, rispondi in formato testuale Markdown.
                                              """},
                {"role": "user", "content": [
                    {"type": "text", "text": f"Questa è la trascrizione dell'audio: {transcription.text}"}
                    ],
                }
                ],
                temperature=0,
                )

print(response.choices[0].message.content)

# Riepilogo OpenAI Dev Day

Durante il primo OpenAI Dev Day, sono stati annunciati diversi nuovi sviluppi e funzionalità:

1. **GPT-4 Turbo**:
   - Supporta fino a 128.000 token di contesto.
   - Introduzione della modalità JSON per risposte valide in formato JSON.
   - Miglioramenti nella capacità di seguire istruzioni e chiamare più funzioni contemporaneamente.
   - Conoscenza aggiornata fino ad aprile 2023.
   - Disponibile nell'API insieme a Dolly 3, GPT-4 Turbo con Vision e il nuovo modello Text-to-Speech.
   - Costo ridotto rispetto a GPT-4 (3x meno per i token di prompt e 2x meno per i token di completamento).

2. **Recupero di informazioni**:
   - Possibilità di integrare conoscenze da documenti esterni o database nelle applicazioni.

3. **Modelli personalizzati**:
   - Programma per creare modelli personalizzati con l'aiuto dei ricercatori di OpenAI.

4. **Limiti di token aumentati**:
   - Raddoppio dei token per minuto per i clienti GPT-4 esistenti.
   - Possibilità di richie